Text Classification Using ALbert

In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.2 MB/s eta 0:00:00


In [3]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.7 MB/s eta 0:00:00


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

## important libraries

In [29]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
import requests
from io import BytesIO
import tarfile

from transformers import AutoTokenizer, AlbertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
import string
import re

In [6]:
from datasets import load_dataset

In [82]:
train_data,test_data = load_dataset("SetFit/tweet_sentiment_extraction",split='train[:50%]'),load_dataset("SetFit/tweet_sentiment_extraction",split='test[:50%]')


(Dataset({
     features: ['textID', 'text', 'label', 'label_text'],
     num_rows: 13740
 }),
 Dataset({
     features: ['text', 'label'],
     num_rows: 12500
 }))

In [83]:
train_df=pd.DataFrame(train_data)
test_df=pd.DataFrame(test_data)
train_df

,textID,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative
3,9642c003ef,what interview! leave me alone,0,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",0,negative
...,...,...,...,...
13735,5b65d15956,http://twitpic.com/676hv - R.I.P Fristy just t...,0,negative
13736,f8afc0cc2e,one thing is good enough for me. friendship r...,2,positive
13737,1a41c7fa70,yep trying to beat the cold winter. Morning s...,2,positive
13738,d388f4b3e6,"Okay, now I`m listening to Jonathan Coulton an...",1,neutral


remove columns that we dont need

In [84]:
train_data=train_df[['text','label']]
train_data
test_data=test_df[['text','label']]
test_data

,text,label
0,Last session of the day http://twitpic.com/67ezh,1
1,Shanghai is also really exciting (precisely -...,2
2,"Recession hit Veronique Branquinho, she has to...",0
3,happy bday!,2
4,http://twitpic.com/4w75p - I like it!!,2
...,...,...
1762,They open at 9 a.m. And I hope to be there ...,1
1763,Is at work.... boo! The Pin-Ups will be play...,1
1764,has just got out the shower,1
1765,he`s back! meeting him in 40 mins,1


In [60]:
# train_data[['label','text']]
# # test_data = test_data[['text', 'label']]


Cleaning the data

In [85]:
wl = WordNetLemmatizer()
ps = PorterStemmer()
nltk.download('stopwords')
sp = set(stopwords.words('english'))
def clean_text(texts):
    final_tmp=[]
    for text in texts:
        #lower and remove punctuation
        text=text.translate(str.maketrans('','',string.punctuation)).lower()
        
        #Remove Hyperlinks
        text=re.sub(r'http\S+','',text)
        
        #Remove numbers and words containing numbers
        text=' '.join([i for i in text.split() if i.isalpha()])
        
        #Now stop words
        text=' '.join(i for i in text.split() if i not in sp)
        
        final_tmp.append(text)
    return final_tmp

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
train_data['text'] = clean_text(train_data['text'])
test_data['text'] = clean_text(test_data['text'])

<ipython-input-86-84dd5f1d107e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['text'] = clean_text(train_data['text'])
<ipython-input-86-84dd5f1d107e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['text'] = clean_text(test_data['text'])


In [87]:
train_data.head()

,text,label
0,id responded going,1
1,sooo sad miss san diego,0
2,boss bullying,0
3,interview leave alone,0
4,sons couldnt put releases already bought,0


In [93]:
test_data.head()

,text,label
0,last session day,1
1,shanghai also really exciting precisely skyscr...,2
2,recession hit veronique branquinho quit compan...,0
3,happy bday,2
4,like,2


In [94]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [95]:
class tweet_review(Dataset):
    def __init__(self, texts, label):
        self.texts = texts
        self.label = label

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        lab = self.label[idx]
        encoding = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(lab, dtype=torch.long)
        }


In [96]:
train_dataset = tweet_review(train_data['text'], train_data['label'])
test_dataset = tweet_review(test_data['text'], test_data['label'])

Create DAta Loaders

In [97]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

Model Initialization

In [98]:
# create model
model = AlbertForSequenceClassification.from_pretrained('albert-base-v2',
                                                      num_labels=len(train_data.label.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [99]:
print(train_dataset.texts)

0                                       id responded going
1                                  sooo sad miss san diego
2                                            boss bullying
3                                    interview leave alone
4                 sons couldnt put releases already bought
                               ...                        
13735                              rip fristy thinking abt
13736            one thing good enough friendship retained
13737    yep trying beat cold winter morning swim alway...
13738       okay im listening jonathan coulton working woo
13739                              cant find tennis racket
Name: text, Length: 13740, dtype: object


In [100]:
## data schedular and optimizer
epochs = 5
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, eps=1e-8)
total_steps = len(train_loader) * 5
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=total_steps, gamma=0.9)

Train The Model

In [101]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input = batch["input_ids"].to(device)
        att_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input, attention_mask=att_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

Epoch 1/5 - Avg Loss: 1.0412
Epoch 2/5 - Avg Loss: 0.9513
Epoch 3/5 - Avg Loss: 0.8848
Epoch 4/5 - Avg Loss: 0.7806
Epoch 5/5 - Avg Loss: 0.6751


## Testing

In [102]:
# testing

model.eval()
predictions = []
val = []
with torch.no_grad():
  for batch in test_loader:
    input = batch["input_ids"].to(device)
    att_mask = batch["attention_mask"].to(device)
    labels = batch["label"].to(device)
    outputs = model(input, attention_mask=att_mask)

    logits = outputs.logits
    _, pred = torch.max(logits, dim=1)

    predictions.extend(pred.detach().cpu().numpy())
    val.extend(labels.detach().cpu().numpy())


acc = accuracy_score(val, predictions)
classification_rep = classification_report(val, predictions)
print(f"Accuracy: {acc:.4f}")
print(classification_rep)

Accuracy: 0.6095
              precision    recall  f1-score   support

           0       0.75      0.32      0.45       497
           1       0.55      0.75      0.64       722
           2       0.65      0.69      0.67       548

    accuracy                           0.61      1767
   macro avg       0.65      0.59      0.59      1767
weighted avg       0.64      0.61      0.59      1767

